In [ ]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
from googlesearch import search

In [ ]:
companies = pd.read_csv("training_data.csv", header = None)
companies = companies[0].tolist()

In [ ]:
def get_soup(url):
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    soup = BeautifulSoup(response, "html.parser")
    return(soup)

In [ ]:
def get_first_url(query):
    for url in search(query, tld="com.sg", num=1, stop=1, pause=3):
        return url

In [ ]:
def get_address(companies):
    company_details = {}
    for company in companies:
        company_details[company] = {}
        query = company + " pitchbook"
        url=get_first_url(query)
        if url[:38] == "https://pitchbook.com/profiles/company":
            soup = get_soup(url)
            address=[]
            try:
                for line in soup.findAll("li", attrs={"class": "info-item__value"}):
                    add = line.text.strip()
                    address.append(add)
                    addr=", ".join([str(x) for x in address])
                print(company, address)
                company_details[company]["Address"]= addr
            except:
                company_details[company]["Address"]= "NA"
                company_details[company]["Phone"]= "NA"
                continue
        else:
            query = company+" bloomberg snapshot"
            url=get_first_url(query)
            if url[:58] == "https://www.bloomberg.com/research/stocks/private/snapshot":
                soup = get_soup(url)
                address = []
                try:
                    add_line = soup.find("div", attrs={"class": "detailsDataContainerLt"})
                    addr=add_line.find("div", attrs={"itemprop": "address"})
                    for p in addr.findAll("p"):
                        add=p.text.strip()
                        address.append(add)
                    print(company, address)
                    addr=", ".join([str(x) for x in address])
                    print(company, addr)
                    phone = []
                    ph_head = soup.find("div", attrs={"class": "detailsDataContainerRt"})
                    ph = ph_head.find("p", attrs={"itemprop": "telephone"})
                    phone = ph.text.strip()
                    print(phone)
                    company_details[company]["Address"]= addr
                    company_details[company]["Phone"]=phone
                except:
                    company_details[company]["Address"]= "NA"
                    company_details[company]["Phone"]= "NA"
                continue
            
            else:
                print(company, "NA") 
                company_details[company]["Address"]= "NA"
    return company_details

In [ ]:
get_address(companies)